In [1]:
import numpy as np
import bqplot
import ipywidgets
import pandas as pd
import matplotlib.colors as mpl_colors
import datetime as dt
%matplotlib inline

## Write-up
Before creating the dashboard, we need to firstly do data cleaning and preprocessing. Assuming the values in variables regarding to 'year' should be treated as invalid if less than 1000, the corresponding records will be removed from the data. There are originally 8862 rows in the "building_inventory.csv". After removing records with invalid values, there are 8544 rows left.

In [2]:
#read in the building_inventory.csv
df = pd.read_csv('/Users/cy/Documents/UIUC_MSIM/IS590DV/week06/building_inventory.csv', header = 0)
df.shape

(8862, 22)

In [3]:
#remove the ourlier and missing values in year relevant variables
years_not_missing = (df['Year Constructed'] > 1000) & (df['Year Acquired'] > 1000)

#make a copy of the orginal read-in csv in order to add new columns to the data frame
inventory = df[years_not_missing].copy()
inventory.shape

(8544, 22)

## Write-up
Since the variables of interests are total square footage across unique and different combination of congressional district and governmental department (Agency Name) over acquired years, I would like to aggregate the data and sum the total square footage grouped by congressional district and governmental department (Agency Name) and acquired years. In order to get the sense of range of acquired year and the corresponding square footage, I also further group the data by unique acquired year.

Since the Gridheat map method of bqplot need use the Matrix or cross tables of the variables of interests, we use the panda crosstab method to create a crosstab, whose row indexes are the values of congressional district, and columns are values of governmental department, and values in cells are the corresponding square footage acquired of a given combination of congressional district and governmental department.

In [4]:
df_sum1 = inventory.groupby(['Congressional Full Name', 'Agency Name','Year Acquired'])['Square Footage'].sum().reset_index()
df_sum1

,Congressional Full Name,Agency Name,Year Acquired,Square Footage
0,Adam Kinzinger,Appellate Court / Third District,1858,15000
1,Adam Kinzinger,Appellate Court / Third District,1860,3700
2,Adam Kinzinger,Department of Central Management Services,1955,17600
3,Adam Kinzinger,Department of Central Management Services,1957,10080
4,Adam Kinzinger,Department of Central Management Services,2000,16450
...,...,...,...,...
2426,Tammy Duckworth,Department of Transportation,2002,20664
2427,Tammy Duckworth,Department of Transportation,2003,150000
2428,Tammy Duckworth,Department of Transportation,2004,520
2429,Tammy Duckworth,Department of Transportation,2005,2880


In [5]:
df_sum2 = inventory.groupby(['Year Acquired'])['Square Footage'].sum().reset_index()
df_sum2

,Year Acquired,Square Footage
0,1753,1200
1,1802,4440
2,1810,4033
3,1832,120000
4,1837,10302
...,...,...
166,2015,305093
167,2016,304839
168,2017,6720
169,2018,17160


In [6]:
con_dep_cross = pd.crosstab(index=df_sum1['Congressional Full Name'],
                           columns = df_sum1['Agency Name'],
                           values = df_sum1['Square Footage'],
                           aggfunc='sum')
con_dep_cross_array = con_dep_cross.to_numpy()
con_dep_cross_array = np.nan_to_num(con_dep_cross)


## Write-up
Since we have the data prepared, now I start to create the dashboard. Basicallly there are 6 steps. 

The first step is to create a ipywidgets label, which is used to print out the value in square footage of selected combo. 

The second step is pre define the elements needed for the Gridheatmap figure. It includes three sub steps, that is setting up the scales, axies and marks. 

The third step is pre define the elements needed for the Line plot. It also includes three sub steps similar with the above.

The fourth step is defining a function used to link the Gridheatmap and line plot together. And for each selected area, it has been limited to only 1 unit square, that is only one unique combination of congressional district and governmental department being able to be selected. And the right figure will show the corresponding trend over a static years range from 1750 to 2020.

The fifth step is creating the 2 figures for the Gridheatmap and line plots, as well as setting up the layout style of those two figures, such as width. 

The last step is putting all above together and finalizing the dashboard through the Hbox and Vbox methods of ipywidgets. Among the two methods, Hbox is used to present the label square footage value of selected combination, and the Vbox is used to show the two figures. 


In [7]:
#1.Create Label
mySelectedLabel = ipywidgets.Label()

In [8]:
#2.Create Heatmap Elements

##scales
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()
col_sc = bqplot.ColorScale(scheme = 'BuPu')

##set up axis
x_ax = bqplot.Axis(scale=x_sc, label='Governmental Department',tick_rotate = 90)
y_ax = bqplot.Axis(scale=y_sc, label='Congressional District', 
                  orientation='vertical')
c_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
##mark
heat_map = bqplot.GridHeatMap(color = con_dep_cross_array ,
                             row = con_dep_cross.index.tolist(),
                             column = con_dep_cross.columns.tolist(),
                             scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
                             interactions = {'click':'select'},
                             anchor_style={'fill':'red'})

In [9]:
#3.Create Line Plot Elements

##scales
x_scl = bqplot.LinearScale(min=1750, max=2020)
y_scl = bqplot.LinearScale()

##axis
ax_xcl = bqplot.Axis(label='Year Acquired', scale = x_scl, tick_rotate = 90)
ax_ycl = bqplot.Axis(label='Square Footage', scale = y_scl,
                    orientation = 'vertical', side='left')

##marks
line_plot = bqplot.Lines(x=df_sum2['Year Acquired'],
                        y=df_sum2['Square Footage'],
                        scales = {'x':x_scl, 'y':y_scl})


In [10]:
#4.Linking Line Plot with Heatmap
def get_data_value(change):
    # to make sure only support single selections
    if len(change['owner'].selected) == 1:
        print(change['owner'].selected[0])
        i,j = change['owner'].selected[0] #i:row_num, j:column_num
        v= con_dep_cross_array[i,j]
        #print(v)
        cong_dist = con_dep_cross.index[i]
        department = con_dep_cross.columns[j]
                
        mySelectedLabel.value = 'Total Square Foot of '+ department +' in ' + cong_dist + ' = ' + str(v)
        
        region_mask = ((df_sum1['Congressional Full Name'] == cong_dist) & 
                       (df_sum1['Agency Name'] == department))
        line_plot.x = df_sum1['Year Acquired'][region_mask]
        line_plot.y = df_sum1['Square Footage'][region_mask]
        
heat_map.observe(get_data_value, 'selected')

In [11]:
#5.Create figs for each figures.
fig_heatmap = bqplot.Figure(marks = [heat_map], axes=[c_ax, y_ax, x_ax])
fig_year = bqplot.Figure(marks=[line_plot], axes=[ax_xcl, ax_ycl])
fig_heatmap.layout.min_width = '600px'
fig_year.layout.min_width = '400px'

In [12]:
#6.Put all together
plots = ipywidgets.HBox([fig_heatmap, fig_year])
myDashboard = ipywidgets.VBox ([mySelectedLabel, plots])
myDashboard

[3 7]
